In [1]:
%cd ..

/users/5/dever120/PyGRANSO


# Trace Optimization

Trace optimization with orthogonal constraints taken from: Effrosini Kokiopoulou, Jie Chen, and Yousef Saad. "Trace optimization and eigenproblems in dimension reduction methods." Numerical Linear Algebra with Applications 18.3 (2011): 565-602.

## Problem Description
Given a symmetric matrix $A$ of dimension $n\times n$, and an arbitrary unitary matrix $V$ of dimension $n\times d$. 

The trace of $V^TAV$ is maximized when $V$ is an orthogonal basis of the eigenspace associated with the (algebraically) largest eigenvalues.

If eigenvalues are labeled decreasingly and $u_1,...,u_d$ are eigenvectors associated with the first $d$ eigenvalues $\lambda_1,...,\lambda_d$, and $U = [u_1,...,u_d]$ with $U^TU=I$, then,

$$\max_{V \in R^{n\times d}, V^TV=I} \text{Tr}[V^TAV]=\text{Tr}[U^TAU]=\lambda_1+...+\lambda_d$$


## Modules Importing
Import all necessary modules and add PyGRANSO src folder to system path.

In [2]:
import time
import torch
import sys

## Adding PyGRANSO directories. Should be modified by user
sys.path.append("/users/5/dever120/PyGRANSO")
from pygranso.pygranso import pygranso
from pygranso.pygransoStruct import pygransoStruct

## Data Initialization 
Specify torch device, and generate data

Use GPU for this problem. If no cuda device available, please set *device = torch.device('cpu')*

In [3]:
device = torch.device("cpu")
n = 5
d = 1
torch.manual_seed(0)

# All the user-provided data (vector/matrix/tensor) must be in torch tensor format.
# As PyTorch tensor is single precision by default, one must explicitly set `dtype=torch.double`.
# Also, please make sure the device of provided torch tensor is the same as opts.torch_device.
A = torch.randn(n, n).to(device=device, dtype=torch.double)
A = (A + A.T) / 2
L, U = torch.linalg.eig(A)
L, U = L.to(dtype=torch.double), U.to(dtype=torch.double)
index = torch.argsort(L, descending=True)
U = U[:, index[0:d]]

/tmp/ipykernel_2502820/2202218492.py:12: UserWarning: Casting complex values to real discards the imaginary part (Triggered internally at /pytorch/aten/src/ATen/native/Copy.cpp:307.)
  L, U = L.to(dtype=torch.double), U.to(dtype=torch.double)


## Function Set-Up

Encode the optimization variables, and objective and constraint functions.

Note: please strictly follow the format of comb_fn, which will be used in the PyGRANSO main algortihm.

In [4]:
# variables and corresponding dimensions.
var_in = {"V": [n, d]}


def user_fn(X_struct, A, d):
    V = X_struct.V

    # objective function
    f = -torch.trace(V.T @ A @ V)

    # inequality constraint, matrix form
    ci = None

    # equality constraint
    ce = pygransoStruct()
    ce.c1 = V.T @ V - torch.eye(d).to(device=device, dtype=torch.double)

    return [f, ci, ce]


comb_fn = lambda X_struct: user_fn(X_struct, A, d)

## User Options
Specify user-defined options for PyGRANSO

In [5]:
opts = pygransoStruct()
opts.torch_device = device
opts.print_frequency = 1
opts.maxit = 3000

## Main Algorithm

In [6]:
start = time.time()
cpu_soln = pygranso(var_spec=var_in, combined_fn=comb_fn, user_opts=opts)
end = time.time()
print("Total Wall Time: {}s".format(end - start))

V = torch.reshape(cpu_soln.final.x, (n, d))

rel_dist = torch.norm(V @ V.T - U @ U.T) / torch.norm(U @ U.T)
print("torch.norm(V @ V.T - U @ U.T) / torch.norm(U @ U.T) = {}".format(rel_dist))

print("torch.trace(V.T @ A @ V) = {}".format(torch.trace(V.T @ A @ V)))
print("torch.trace(U.T @ A @ U) = {}".format(torch.trace(U.T @ A @ U)))
print("sum of first d eigvals = {}".format(torch.sum(L[index[0:d]])))
print("sorted eigs = {}".format(L[index]))



╔═════ QP SOLVER NOTICE ════════════════════════════════════════════════════════════════════════╗
║  PyGRANSO requires a quadratic program (QP) solver that has a quadprog-compatible interface,  ║
║  the default is osqp. Users may provide their own wrapper for the QP solver.                  ║
║  To disable this notice, set opts.quadprog_info_msg = False                                   ║
╚═══════════════════════════════════════════════════════════════════════════════════════════════╝
═════════════════════════════════════════════════════════════════════════════════════════════════════════════════╗
PyGRANSO: A PyTorch-enabled port of GRANSO with auto-differentiation                                             ║ 
Version 1.2.0                                                                                                    ║ 
Licensed under the AGPLv3, Copyright (C) 2021-2022 Tim Mitchell and Buyun Liang                                  ║ 
═════════════════════════════════════════════

## CUDA Computations

In [7]:
device = torch.device("cuda")
n = 5
d = 1
torch.manual_seed(0)

# All the user-provided data (vector/matrix/tensor) must be in torch tensor format.
# As PyTorch tensor is single precision by default, one must explicitly set `dtype=torch.double`.
# Also, please make sure the device of provided torch tensor is the same as opts.torch_device.
A = torch.randn(n, n).to(device=device, dtype=torch.double)
A = (A + A.T) / 2
L, U = torch.linalg.eig(A)
L, U = L.to(dtype=torch.double), U.to(dtype=torch.double)
index = torch.argsort(L, descending=True)
U = U[:, index[0:d]]

In [8]:
# variables and corresponding dimensions.
var_in = {"V": [n, d]}


def user_fn(X_struct, A, d):
    V = X_struct.V

    # objective function
    f = -torch.trace(V.T @ A @ V)

    # inequality constraint, matrix form
    ci = None

    # equality constraint
    ce = pygransoStruct()
    ce.c1 = V.T @ V - torch.eye(d).to(device=device, dtype=torch.double)

    return [f, ci, ce]


comb_fn = lambda X_struct: user_fn(X_struct, A, d)

In [9]:
opts = pygransoStruct()
opts.torch_device = device
opts.print_frequency = 1
opts.maxit = 3000

In [10]:
start = time.time()
cuda_soln = pygranso(var_spec=var_in, combined_fn=comb_fn, user_opts=opts)
end = time.time()
print("Total Wall Time: {}s".format(end - start))

V = torch.reshape(cuda_soln.final.x, (n, d))

rel_dist = torch.norm(V @ V.T - U @ U.T) / torch.norm(U @ U.T)
print("torch.norm(V @ V.T - U @ U.T) / torch.norm(U @ U.T) = {}".format(rel_dist))

print("torch.trace(V.T @ A @ V) = {}".format(torch.trace(V.T @ A @ V)))
print("torch.trace(U.T @ A @ U) = {}".format(torch.trace(U.T @ A @ U)))
print("sum of first d eigvals = {}".format(torch.sum(L[index[0:d]])))
print("sorted eigs = {}".format(L[index]))



╔═════ QP SOLVER NOTICE ════════════════════════════════════════════════════════════════════════╗
║  PyGRANSO requires a quadratic program (QP) solver that has a quadprog-compatible interface,  ║
║  the default is osqp. Users may provide their own wrapper for the QP solver.                  ║
║  To disable this notice, set opts.quadprog_info_msg = False                                   ║
╚═══════════════════════════════════════════════════════════════════════════════════════════════╝
═════════════════════════════════════════════════════════════════════════════════════════════════════════════════╗
PyGRANSO: A PyTorch-enabled port of GRANSO with auto-differentiation                                             ║ 
Version 1.2.0                                                                                                    ║ 
Licensed under the AGPLv3, Copyright (C) 2021-2022 Tim Mitchell and Buyun Liang                                  ║ 
═════════════════════════════════════════════

## Testing

In [13]:
torch.testing.assert_close(
    cpu_soln.final.x,
    cuda_soln.final.x.cpu(),
    atol=1e-4,
    rtol=1e-4,
)

In [14]:
cpu_soln.final.x

tensor([[ 0.0561],
        [ 0.1102],
        [-0.3954],
        [-0.8338],
        [ 0.3648]], dtype=torch.float64)

In [15]:
cuda_soln.final.x.cpu()

tensor([[ 0.0560],
        [ 0.1102],
        [-0.3954],
        [-0.8338],
        [ 0.3648]], dtype=torch.float64)